In [4]:
from lcm.utils.checkpoint import load_from_json_checkpoint
import seaborn as sns
import pandas as pd
import torch

from lcm.trainers import CompartmentalTrainer
from lcm.utils.visualization import parse_date

import matplotlib.pyplot as plt
import datetime

In [63]:
exps_path = "./giordano_confrontation"
giordano_path = os.path.join(exps_path, "giordano.json")
nophichi_path = os.path.join(exps_path, "nophichi.json")
phichi_path = os.path.join(exps_path, "phichi.json")

giordano, dataset = load_from_json_checkpoint(giordano_path)
nophichi, _ = load_from_json_checkpoint(nophichi_path)
phichi, _ = load_from_json_checkpoint(phichi_path)

dataset_size = dataset.train_size + dataset.val_size + dataset.test_size

In [9]:
# create trainer

giordano_exp = CompartmentalTrainer(
    dataset=dataset,
    model=giordano,
    uuid_prefix="all_regions", uuid="",
)

nophichi_exp = CompartmentalTrainer(
    dataset=dataset,
    model=nophichi,
    uuid_prefix="all_regions", uuid="",
)

phichi_exp = CompartmentalTrainer(
    dataset=dataset,
    model=phichi,
    uuid_prefix="all_regions", uuid="",
)

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
GPU available: True, used: False
TPU available: None, using: 0 TPU cores
GPU available: True, used: False
TPU available: None, using: 0 TPU cores


In [16]:
giordano_out = giordano_exp.test(giordano, dataset.test_dataloader(), verbose=False)[0]
nophichi_out = nophichi_exp.test(nophichi, dataset.test_dataloader(), verbose=False)[0]
phichi_out = phichi_exp.test(phichi, dataset.test_dataloader(), verbose=False)[0]

Testing: 100%|██████████| 1/1 [00:02<00:00,  2.41s/it]


In [55]:
giordano_val = giordano_out["val_loss_unweighted"]
nophichi_val = nophichi_out["val_loss_unweighted"]
phichi_val = phichi_out["val_loss_unweighted"]


outs = [giordano_out, nophichi_out, phichi_out]
models = ["Giordano", "LCM", "LCM+"]
targets = ["d","r","t","h_detected","e", "unweighted"]
giordano_out.keys()

dict_keys(['hats', 'targets', 'target_loss', 'regularization_loss', 'val_loss_d', 'val_loss_r', 'val_loss_t', 'val_loss_h_detected', 'val_loss_e', 'val_loss_weighted', 'val_loss_unweighted'])

In [57]:
# print table with val loss

def print_table(outs, row_printer):
    print(r"""
\begin{tabular}{||c c c c c c c||} 
    \hline
    \textbf{Model} & \textbf{D} & \textbf{R} & \textbf{T} & \textbf{H_d} & \textbf{E} & \textbf{Total}\\ [0.5ex] 
    \hline""")

    for out, model_name in zip(outs, models): 
        row_printer(out, model_name)
        print("    \\hline")

    print("\\end{tabular}")

In [59]:
print("TABLE WITH ABSOLUTE VALUES")

def print_row(out, model_name):

    values = " & ".join([f"{out['val_loss_'+key]:.2f}" for key in targets])
    print(f"    {model_name} & {values}")

print_table(outs, print_row)

TABLE WITH ABSOLUTE VALUES

\begin{tabular}{||c c c c c c c||} 
    \hline
    \textbf{Model} & \textbf{D} & \textbf{R} & \textbf{T} & \textbf{H_d} & \textbf{E} & \textbf{Total}\\ [0.5ex] 
    \hline
    Giordano & 19973.70 & 1764.75 & 3019.99 & 1860.53 & 8797.16 & 35416.14
    \hline
    LCM & 13609.48 & 4665.04 & 1198.25 & 788.52 & 7516.44 & 27777.74
    \hline
    LCM+ & 12243.39 & 3358.04 & 1134.33 & 712.20 & 4180.89 & 21628.85
    \hline
\end{tabular}


In [60]:
print("TABLE WITH MEAN VALUES")

def print_row(out, model_name):

    values = " & ".join([f"{out['val_loss_'+key] / dataset.test_size:.2f}" for key in targets])
    print(f"    {model_name} & {values}")

print_table(outs, print_row)

TABLE WITH MEAN VALUES

\begin{tabular}{||c c c c c c c||} 
    \hline
    \textbf{Model} & \textbf{D} & \textbf{R} & \textbf{T} & \textbf{H_d} & \textbf{E} & \textbf{Total}\\ [0.5ex] 
    \hline
    Giordano & 1426.69 & 126.05 & 215.71 & 132.89 & 628.37 & 2529.72
    \hline
    LCM & 972.11 & 333.22 & 85.59 & 56.32 & 536.89 & 1984.12
    \hline
    LCM+ & 874.53 & 239.86 & 81.02 & 50.87 & 298.63 & 1544.92
    \hline
\end{tabular}


In [78]:
from lcm.losses.target_losses import MAPE
nrmse = giordano.loss_fn
giordano.loss_fn = MAPE(nrmse.weights)
nophichi.loss_fn = MAPE(nrmse.weights)
phichi.loss_fn = MAPE(nrmse.weights)

giordano_mape_out = giordano_exp.test(giordano, dataset.test_dataloader(), verbose=False)[0]
nophichi_mape_out = giordano_exp.test(nophichi, dataset.test_dataloader(), verbose=False)[0]
phichi_mape_out = giordano_exp.test(phichi, dataset.test_dataloader(), verbose=False)[0]

Testing: 100%|██████████| 1/1 [00:02<00:00,  2.39s/it]


In [83]:
print("TABLE WITH MAPES VALUES")

outs = [giordano_mape_out, nophichi_mape_out, phichi_mape_out]
def print_row(out, model_name):

    values = " & ".join([f"{out['val_loss_'+key]*100:.2f}%" for key in targets])
    print(f"    {model_name} & {values}")

print_table(outs, print_row)

TABLE WITH MAPES VALUES

\begin{tabular}{||c c c c c c c||} 
    \hline
    \textbf{Model} & \textbf{D} & \textbf{R} & \textbf{T} & \textbf{H_d} & \textbf{E} & \textbf{Total}\\ [0.5ex] 
    \hline
    Giordano & 37.57% & 7.93% & 130.32% & 5.78% & 61.72% & 243.33%
    \hline
    LCM & 24.20% & 23.09% & 51.38% & 2.76% & 52.61% & 154.05%
    \hline
    LCM+ & 21.59% & 16.28% & 48.06% & 2.61% & 26.56% & 115.09%
    \hline
\end{tabular}
